# Phase 3: Data Preprocessing & Feature Engineering

- Load raw train data
- Clean and transform
- Engineer key features (time-based, frequency, distance, amount, age)
- Handle PII ethically
- Save processed data

## Imports & Path Set-up

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from math import radians, sin, cos, sqrt, atan2

RAW_DATA_PATH = '../data/raw/'
PROCESSED_DATA_PATH = '../data/processed/'

os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

train_file = os.path.join(RAW_DATA_PATH, 'fraudTrain.csv')
sample_file = os.path.join(RAW_DATA_PATH, 'fraudTrain_sample_10k.csv')

# For faster development: use sample first, then switch to full train
USE_SAMPLE = True  # Change to False later for full data
data_file = sample_file if USE_SAMPLE else train_file

print("Using file:", data_file)

Using file: ../data/raw/fraudTrain_sample_10k.csv


## Load data & Initial cleaning


In [3]:
# Load the CSV data into a DataFrame (skip unnamed index if present)
df = pd.read_csv(data_file, index_col=0)

# Print basic info for verification
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing values:\n", df.isnull().sum())

# Convert transaction time column to datetime for easier extraction (e.g., hour, month)
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Convert date of birth to datetime for age calculation
df['dob'] = pd.to_datetime(df['dob'])

# Drop unnecessary/redundant columns (e.g., unique IDs not useful for modeling)
cols_to_drop = ['trans_num', 'unix_time']
df = df.drop(columns=cols_to_drop, errors='ignore')

# Ethical handling: Drop direct PII columns to avoid privacy risks; we'll compute age but consider excluding it from models
pii_cols = ['first', 'last', 'street', 'city', 'state', 'zip', 'job']
df = df.drop(columns=pii_cols, errors='ignore')

print("\nAfter cleaning shape:", df.shape)
df.head(3)

Shape: (10000, 22)

Columns: ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']

Missing values:
 trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

After cleaning shape: (10000, 13)


,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
Unnamed: 0,,,,,,,,,,,,,
1045211,2020-03-09 15:09:26,577588686219,fraud_Towne LLC,misc_pos,194.51,M,40.6153,-79.4545,972,1997-10-23,40.420453,-78.865012,0
547406,2019-08-22 15:49:01,30376238035123,fraud_Friesen Ltd,health_fitness,52.32,F,42.8250,-124.4409,217,1928-10-01,42.758860,-123.636337,0
110142,2019-03-04 01:34:16,4658490815480264,fraud_Mohr Inc,shopping_pos,6.53,F,39.9636,-79.7853,184,1945-11-04,40.475159,-78.898190,0


## Basic time features

In [4]:
# Extract hour of day from transaction time (for patterns like overnight fraud spikes)
df['trans_hour'] = df['trans_date_trans_time'].dt.hour

# Extract day of week (0=Monday, 6=Sunday; for weekend patterns)
df['trans_dayofweek'] = df['trans_date_trans_time'].dt.dayofweek

# Extract month (for January/holiday seasonality; fraud spikes in Jan post-holidays)
df['trans_month'] = df['trans_date_trans_time'].dt.month

# Research-inspired: Flag late-night hours (midnight-3am, where fraud often spikes due to low monitoring)
df['is_late_night'] = df['trans_hour'].apply(lambda x: 1 if (0 <= x <= 3) else 0)

# Broader night flag (21:00-05:00, as fallback)
df['is_night'] = df['trans_hour'].apply(lambda x: 1 if (x >= 21 or x < 5) else 0)

# Holiday season flag (Nov-Dec for Black Friday/Cyber Monday spikes)
df['is_holiday_season'] = df['trans_month'].apply(lambda x: 1 if x in [11, 12] else 0)

## Distance between user & merchant (Strong fraud signal)

In [5]:
# Haversine function to calculate km distance (accounts for Earth's curve; fraud often far from user location)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Apply to each row to create distance feature
df['distance_km'] = df.apply(
    lambda row: haversine(
        row['lat'], row['long'],
        row['merch_lat'], row['merch_long']
    ), axis=1
)

# Quick stats to verify (distances should range 0-1000+ km)
print(df['distance_km'].describe())

count    10000.000000
mean        76.243499
std         29.156472
min          0.693570
25%         55.284126
50%         78.591084
75%         98.594250
max        146.805829
Name: distance_km, dtype: float64


## Time-based frequency & recency per card

In [7]:
# Sort by card number and transaction time (required for groupby diff and cumcount)
df = df.sort_values(['cc_num', 'trans_date_trans_time'])

# Time since last transaction (in hours) - this part is already correct
df['time_since_last_trans'] = df.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds() / 3600
df['time_since_last_trans'] = df['time_since_last_trans'].fillna(0)  # First transaction gets 0

# For rolling frequency (count of transactions in last N days per card)
# Method: Use cumcount() + time delta filter (approximation, fast and reliable)

# Days since first transaction for each card
df['days_since_first'] = (df['trans_date_trans_time'] - 
                          df.groupby('cc_num')['trans_date_trans_time'].transform('min')).dt.days

# Approximate frequency in last 30 days (number of prior transactions within 30 days)
# This is a fast proxy using cumcount + condition
df['freq_last_30d'] = df.groupby('cc_num').cumcount().where(df['days_since_first'] <= 30, 0)

# Same for 7 days
df['freq_last_7d'] = df.groupby('cc_num').cumcount().where(df['days_since_first'] <= 7, 0)

# Optional: last 1 day (very recent velocity)
df['freq_last_1d'] = df.groupby('cc_num').cumcount().where(df['days_since_first'] <= 1, 0)

# Print quick check
print("Frequency features sample:\n", df[['cc_num', 'trans_date_trans_time', 
                                          'time_since_last_trans', 'freq_last_30d', 
                                          'freq_last_7d']].head(10))

Frequency features sample:
                  cc_num trans_date_trans_time  time_since_last_trans  \
Unnamed: 0                                                             
202030      60416207185   2019-04-14 01:03:56               0.000000   
419976      60416207185   2019-07-08 01:44:58            2040.683889   
429216      60416207185   2019-07-11 18:22:01              88.617500   
492100      60416207185   2019-08-03 06:57:14             540.586944   
609144      60416207185   2019-09-15 20:45:29            1045.804167   
635766      60416207185   2019-09-28 00:57:33             292.201111   
643165      60416207185   2019-09-30 14:36:58              61.656944   
649941      60416207185   2019-10-04 03:26:53              84.831944   
728326      60416207185   2019-11-07 14:16:51             826.832778   
865951      60416207185   2019-12-19 02:43:39             996.446667   

            freq_last_30d  freq_last_7d  
Unnamed: 0                               
202030                 

## Amount Transformations

In [8]:
# Log transform amount (handles skewness; fraud often in extreme amounts)
df['log_amt'] = np.log1p(df['amt'])  # log(1 + amt) to avoid log(0)

## Age features based on research (targetted groups between 30 and 39

In [9]:
# Approximate age at transaction (year difference; for patterns like 30-39 targeting or senior losses)
trans_year = df['trans_date_trans_time'].dt.year
birth_year = df['dob'].dt.year
df['age_at_trans'] = trans_year - birth_year

# Bin age for groups (e.g., 30-39 flag; seniors 60+ for higher loss insights)
bins = [0, 20, 30, 40, 50, 60, 120]
labels = ['<20', '20-29', '30-39', '40-49', '50-59', '60+']
df['age_bin'] = pd.cut(df['age_at_trans'], bins=bins, labels=labels)

# Optional flag for 30-39 (highest volume group)
df['is_30_39'] = df['age_bin'].apply(lambda x: 1 if x == '30-39' else 0)

## Reorder columns & save processed data

In [10]:
# Reorder columns for clarity (focus on key ones first)
important_cols = ['trans_date_trans_time', 'cc_num', 'category', 'amt', 'log_amt',
                  'trans_hour', 'is_late_night', 'trans_month', 'is_holiday_season',
                  'distance_km', 'time_since_last_trans', 'freq_last_30d', 'age_at_trans',
                  'age_bin', 'is_fraud']  # Add more as needed

df_processed = df[important_cols + [col for col in df.columns if col not in important_cols]]

# Save the processed DataFrame to CSV
processed_file = os.path.join(PROCESSED_DATA_PATH, 'train_processed.csv')
df_processed.to_csv(processed_file, index=False)
print("Processed data saved to:", processed_file)

# Final verification
print("\nProcessed shape:", df_processed.shape)
df_processed.head(3)

Processed data saved to: ../data/processed/train_processed.csv

Processed shape: (10000, 29)


,trans_date_trans_time,cc_num,category,amt,log_amt,trans_hour,is_late_night,trans_month,is_holiday_season,distance_km,...,city_pop,dob,merch_lat,merch_long,trans_dayofweek,is_night,days_since_first,freq_last_7d,freq_last_1d,is_30_39
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
202030,2019-04-14 01:03:56,60416207185,gas_transport,55.26,4.029984,1,1,4,0,71.499270,...,1645,1986-02-17,43.647774,-108.905541,6,1,0,0,0,1
419976,2019-07-08 01:44:58,60416207185,grocery_net,42.28,3.767691,1,1,7,0,83.558426,...,1645,1986-02-17,43.603961,-108.273157,0,1,85,0,0,1
429216,2019-07-11 18:22:01,60416207185,food_dining,30.73,3.457263,18,0,7,0,61.168308,...,1645,1986-02-17,42.464779,-109.039110,3,0,88,0,0,1
